In [1]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression as LR
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier as GB
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier as KNC
from Database import Database
from numpy import array
from sklearn.naive_bayes import MultinomialNB as NB
import sqlite3
import tqdm
import random

In [2]:
conn = sqlite3.connect('berita.db')
c = conn.cursor()
c.execute("SELECT Text,Date,Clock,Sentiment From Berita ")
result = c.fetchall()

In [4]:
d = Database()
data = {}
label = {}
for I in result:
    session = d.cariSesi(I[1],I[2])[0]
    if session not in data:
        data[session] = []
        label[session] = I[3]
    sentence = I[0]
    newSentence = ""
    for J in sentence:
        if J.isalpha():
            newSentence += J
        else:
            newSentence += " "
    data[session].append(newSentence)

In [5]:
# Sorting the chronological order

#MeTooLazySoMeBubbleSort

chronology = list(data.keys())

for I in range(len(data.keys())):
    for J in range(I+1,len(data.keys())):
        if chronology[I] > chronology[J]:
            chronology[I],chronology[J] = chronology[J],chronology[I]

In [16]:
date = [chronology[len(chronology)*I//6-1] for I in range(1,7)]

In [7]:
def balancedTrain(X,y,mode):
    '''
    mode has two options:
    -> 'CV'
    then balancedTrain would suit it undersampling method for 
    cross validation
    
    -> 'Boosting'
    then balancedTrain would suit is sampling method for
    boosting    
    '''
    assert mode == 'CV' or mode == 'Boosting'
    balancedX = []
    balancedY = []
    
    index = {}
    
    assert len(X) == len(y)
    
    #Mencatat indeks mana yang positif dan mana yang negatif
    for I in range(len(X)):
        if y[I] not in index:
            index[y[I]] = []
        index[y[I]].append(I)
    
    minimumPoint = min([len(I) for I in index.values()])    
    
    #Memastikan jumlah (+) dan (-) sama
    for I in index:
        if mode == 'CV':
            chosen = random.sample(index[I],minimumPoint)
        else:
            chosen = random.choices(index[I],k=minimumPoint)
        for J in chosen:
            balancedX.append(X[J])
            balancedY.append(y[J])
    return balancedX, balancedY

In [8]:
def transform(text):
    listNews = []
    for news in text:
        listSentence = []
        oldText = news
        while True:
            newText = oldText.replace("  "," ")
            if oldText == newText:
                break
            oldText = newText
        listSentence.extend(newText)
        splitText = newText.split(" ")
        listSentence.append(" ".join([splitText[I] for I in range(0,len(splitText),2)]))
        listSentence.append(" ".join([splitText[I] for I in range(1,len(splitText),2)]))
        listNews.append(" SNIP ".join(listSentence))
    return listNews

In [9]:
def splitGroup(kFold):
    #Me too lazy to plug in sklearn
    assert len(X) == len(y)
    index = [int(I) for I in range(len(X))]
    random.shuffle(index)
    group = [index[len(index)*I//kFold:len(index)*(I+1)//kFold] for I in range(kFold)]
    return group

In [10]:
#Custom class for GradientBoosting
class Boosting():
    def __init__(self):
        self.clf = GB()
    def fit(self,X,y):
        self.clf.fit(X,y)
    def predict(self,X):
        m = int(X.shape[0] ** (0.5))
        pred = []
        for I in range(m):
            pred.extend(self.clf.predict(X[I*X.shape[0]//m:(I+1)*X.shape[0]//m].toarray()))
        return pred
#Custom class for K Nearest Neighbor
class Neighbors:
    def __init__(self):
        self.clf = KNC()
    def fit(self,X,y):
        self.clf.fit(X,y)
    def predict(self,X):
        m = int(X.shape[0] ** (0.5))
        pred = []
        for I in range(m):
            pred.extend(self.clf.predict(X[I*X.shape[0]//m:(I+1)*X.shape[0]//m]))
        return pred

In [11]:
def bersihkanTeksBerita(array):
    hasil = []
    for news in array:
        newSentence = ""
        for J in news:
            if J.isalpha():
                newSentence += J
            else:
                newSentence += " "
        hasil.append(newSentence)
    return hasil

In [12]:
clfOption = [Boosting(),LR(n_jobs = -1),NB(),LinearSVC(),Neighbors(),RFC()]
mrePred = []

In [13]:
def mrc(pred,Y):
    assert len(pred)==len(Y)
    pred = array(pred)
    Y    = array(Y)
    
    TP, FP , TN, FN = 0,0,0,0
    
    for I in range(len(pred)):
        if pred[I] == Y[I]:
            if pred[I] == 1:
                TP += 1
            else:
                TN += 1
        else:
            if pred[I] == -1:
                FP += 1
            else:
                FN += 1
    print(TP,FP,TN,FN)
    try:
        return ((TP*TN)-(FP*FN)) / ((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**(0.5)
    except:
        return 0

In [14]:
del result

In [18]:
mrePred = []
for iter in range(5):
    mreTotal = []
    query = "Select * from berita WHERE Date <= "+str(date[iter])+" AND Title LIKE '%ekono%' "
    c.execute(query)
    trainData = c.fetchall()
    
    query = "Select * from berita WHERE Date <= "+str(date[iter])+" AND NOT Title LIKE '%ekono%' "
    c.execute(query)
    trainDataUnknown = c.fetchall()
    
    query = "Select * from berita WHERE Date <= "+str(date[iter+1])+" AND "+str(date[iter])+"< Date AND NOT Title LIKE '%ekono%' "
    c.execute(query)
    testData = c.fetchall()
    
    print("Data berhasil difetch")
    print(len(trainDataUnknown),len(trainData),len(testData))
    filtered = []
    for I in range(0,len(trainDataUnknown),len(trainData)):
        X = [J[3] for J in trainData]
        y = [int(1) for J in trainData]
        toEvaluate = [J[3] for J in trainDataUnknown[I:I+len(trainData)]]
        X += toEvaluate
        y += [int(0) for J in toEvaluate]
    
        counter = CV()
        vector = counter.fit_transform(bersihkanTeksBerita(X))
        toEvaluateVector = counter.transform(bersihkanTeksBerita(toEvaluate))
    
        bayes = NB()
        bayes.fit(vector,y)
        predict = bayes.predict_proba(toEvaluateVector)
    
        for J in range(len(predict)):
            if predict[J][1] > 0.9:
                filtered.append(trainDataUnknown[I+J])
    
    print("Data berhasil difilter")
    
    trainData += filtered
    print(len(filtered))
    # Do cross-validation to choose the best feature selection
    
    X = []
    y = []
    
    for I in trainData:
        X.append(I[3])
        y.append(I[7])
        
    group = splitGroup(5)
    XkFold = [[X[J] for J in K]for K in group]
    YkFold = [[y[J] for J in K]for K in group]
    counterList = []
    selectList = []
    mreTotal = []
  
    for I in range(5):
        xTrain = []
        yTrain = []
    
        xTest  = []
        yTest = []
        for J in range(5):
            if J == I:
                for L in XkFold[J]:
                    xTest.append(L)
                yTest.extend(YkFold[J])
            else:
                for L in XkFold[J]:
                    xTrain.append(L)
                yTrain.extend(YkFold[J])

        xTrain = transform(xTrain)
        xTest = transform(xTest)

        assert len(xTrain) == len(yTrain)
        xTrainNew , yTrainNew = balancedTrain(xTrain,yTrain,'CV')
        counterList.append(CV(ngram_range=(2,2),min_df=5)) 
        trainVector = counterList[-1].fit_transform(xTrainNew)
        testVector  = counterList[-1].transform(xTest)

        selectList.append(SelectKBest(chi2, k = min(10000,trainVector.shape[1])))

        trainVector = selectList[-1].fit_transform(trainVector,yTrainNew)
        testVector  = selectList[-1].transform(testVector)

        mreTotal.append(0)
        for J in clfOption:
            J.fit(trainVector,yTrainNew)
            prediction = J.predict(testVector)
            mreTotal[-1] += mrc(prediction,yTest)
    
    index = mreTotal.index(max(mreTotal))
    
    mrePred.append({'post':[],
                    'chronological':[]})
    
    xTrainNew = []
    yTrainNew = []
    # Generating boosting data
    for I in range(5):
        X_temp , y_temp = balancedTrain(xTrain,yTrain,'Boosting')
        xTrainNew.append(X_temp)
        yTrainNew.append(y_temp)

    trainVector = [counterList[index].transform(I) for I in xTrainNew]        
    trainVector = [selectList[index].transform(I)for I in trainVector]
        
    # Create the test set of chronological entries
        
    lengthOfTestData = {}
    testX = []
    dataY = {}

    for entry in testData:
        if entry[5] not in lengthOfTestData:
            lengthOfTestData[entry[5]] = 0
            dataY[entry[5]] = entry[7]
        lengthOfTestData[entry[5]] += 1
        testX.append(entry[3])
    
    testX = counterList[index].transform(transform(testX))
    testVector = selectList[index].transform(testX)
    print("Mulai training")
    for I in clfOption:
        postPredict = array([int(0) for J in range(testVector.shape[0])])
        for boostingIter in range(5):
            I.fit(trainVector[boostingIter],yTrainNew[boostingIter])
            postPredict += I.predict(testVector)
        postPredict = [[-1,1][J>0] for J in postPredict]

        dayPredict = []
        postY = []
        dayY = []
        sumIndex = 0

        for dateTested in lengthOfTestData:               
            dayPredict.append([-1,1][sum(postPredict[sumIndex:sumIndex+lengthOfTestData[dateTested]])>0])
            postY.extend([dataY[dateTested] for I in range(lengthOfTestData[dateTested])])
            dayY.append(dataY[dateTested])
            sumIndex += lengthOfTestData[dateTested]

        mrePred[-1]['post'].append(mrc(postPredict,postY))
        mrePred[-1]['chronological'].append(mrc(dayPredict,dayY))
            

Data berhasil difetch
17280 811 14377
Data berhasil difilter
1356
170 78 107 78
153 95 124 61
133 115 127 58
0 248 185 0
127 121 98 87
118 130 126 59
179 95 93 66
168 106 95 64
180 94 86 73
0 274 159 0
147 127 74 85
146 128 117 42
134 123 125 52
173 84 117 60
177 80 96 81
257 0 0 177
143 114 84 93
156 101 101 76
139 121 109 64
155 105 100 73
159 101 100 73
1 259 173 0
142 118 82 91
128 132 126 47
139 112 127 56
155 96 131 52
160 91 118 65
0 251 183 0
126 125 93 90
119 132 133 50
Mulai training
3992 6030 2636 1719
19 68 27 9
6735 3287 1483 2872
80 7 2 34
7219 2803 1229 3126
86 1 0 36
10022 0 0 4355
87 0 0 36
5393 4629 1962 2393
60 27 7 29
5643 4379 1967 2388
61 26 13 23
Data berhasil difetch
31657 1766 14396
Data berhasil difilter
3057
446 172 188 158
390 228 206 140
400 218 197 149
374 244 199 147
595 23 19 327
320 298 234 112
448 166 159 192
397 217 203 148
392 222 202 149
0 614 351 0
311 303 160 191
324 290 229 122
484 176 153 151
414 246 182 122
426 234 182 122
660 0 0 304
344 316 1

In [19]:
name = ["Gradient Boosting","Logistic Regression","Naive Bayes","Linear SVC","K nearest neighbor","Random forest"]
for I in range(len(name)):
    chronological = [J['chronological'][I] for J in mrePred]
    post = [J['post'][I] for J in mrePred]
    print("%s -> (chronological) %f (post) %f " % (name[I],sum(chronological)/5,sum(post)/5))

Gradient Boosting -> (chronological) 0.062733 (post) 0.021852 
Logistic Regression -> (chronological) 0.118934 (post) 0.024388 
Naive Bayes -> (chronological) 0.142702 (post) 0.020623 
Linear SVC -> (chronological) 0.003153 (post) 0.000898 
K nearest neighbor -> (chronological) -0.019595 (post) -0.001579 
Random forest -> (chronological) 0.060045 (post) 0.012550 


In [20]:
mrePred

[{'chronological': [-0.03429916122096134,
   -0.04351210017041174,
   -0.058238704909845236,
   0,
   -0.11791289070861165,
   0.06087679614953948],
  'post': [0.0033853842154468613,
   0.012247596968392274,
   0.002577444274549272,
   0,
   -0.01048316037263073,
   0.013627675072789504]},
 {'chronological': [0.02919763719475011,
   0.05443709712337166,
   0.1656166479917091,
   0.015763678553028876,
   0,
   0.01981412661311067],
  'post': [-0.005642207314298505,
   -0.015849389766556973,
   -0.014328298239338105,
   -0.018950864175118622,
   0.0016411002274963236,
   -0.03446067112631721]},
 {'chronological': [0.07938841860374447,
   0.2865642240726484,
   0.1684775569867293,
   0,
   -0.040201858939733416,
   0.080774168684147],
  'post': [0.06473423587312536,
   0.03921391787731933,
   0.03274746053348091,
   0.01397705715640349,
   -0.0033782855491423717,
   0.042910306962324805]},
 {'chronological': [0.17142857312652204,
   0.1877291060740164,
   0.1758246421298328,
   0,
   0.00

In [ ]:
for I in clfOption:
    postPredict = array([int(0) for J in range(testVector.shape[0])])
    for boostingIter in range(5):
        I.fit(trainVector[boostingIter],yTrainNew[boostingIter])
        postPredict += I.predict(testVector)
    postPredict = [[-1,1][J>0] for J in postPredict]
        
    dayPredict = []
    postY = []
    dayY = []
    sumIndex = 0
    
    for date in lengthOfTestData:               
        dayPredict.append([-1,1][sum(postPredict[sumIndex:sumIndex+lengthOfTestData[date]])>0])
        postY.extend([dataY[date] for I in range(lengthOfTestData[date])])
        dayY.append(dataY[date])
        sumIndex += lengthOfTestData[date]
            
    mrePred[-1]['post'].append(mrc(postPredict,postY))
    mrePred[-1]['chronological'].append(mrc(dayPredict,dayY))

In [23]:
print("Mulai training")
    for I in clfOption:
        postPredict = array([int(0) for J in range(testVector.shape[0])])
        for boostingIter in range(5):
            I.fit(trainVector[boostingIter],yTrainNew[boostingIter])
            postPredict += I.predict(testVector)
        postPredict = [[-1,1][J>0] for J in postPredict]
        
        dayPredict = []
        postY = []
        sumIndex = 0
    
        for date in entry:               
            dayPredict.append([-1,1][sum(postPredict[sumIndex:sumIndex+len(lengthOfTestData[date])])>0])
            postY.extend([label[J] for I in range(len(lengthOfTestData[date]))])
            dayY.append(label[J])
            sumIndex += len(data[J])
            
        mrePred[-1]['post'].append(mrc(postPredict,postY))
        mrePred[-1]['chronological'].append(mrc(dayPredict,dayY))

IndentationError: unexpected indent (<ipython-input-23-136562ec89de>, line 2)

In [53]:
trainVector.shape[1]

9013

In [45]:
testX[0]

'\nJAKARTA, Pengamat ekonomi dari Institute for Development of Economics and Finance (Indef) Enny Sri Hartati, mengungkapkan Perum Bulog paling ideal menguasai 5 juta ton beras per tahun.Dengan pasokan sebesar itu, Enny yakin mafia beras akan hilang dari peredaran. "Idealnya Bulog 5 juta ton, benar-benar tidak ada mafia beras melakukan spekulasi," ujar Enny, Senin (1/6/2015). Enny berpendapat minimal Perum Bulog harus bisa menguasai 10 persen atau 4,5 juta ton dari konsumsi masyarakat sebanyak 45 juta ton. Jika dengan pasokan hingga akhir Mei 2015 hanya 1,2 juta ton, mafia beras bisa cepat melakukan spekulasi harga beras. "Bulog minimal kuasai 10 persen atau 4,5 juta ton. Itu belum termasuk raskin," ungkap Enny. Enny memaparkan kebutuhan untuk raskin (beras untuk masyarakat miskin) juga harus dipenuhi. Angka pasokan ideal untuk raskin menurut Enny sebesar 1,6 juta ton per tahun. "Kebutuhan raskin harus ada stok untuk cadangan," papar Enny. (Adiatmaputra Fajar Pratama)'